In [1]:
from queue import Queue
from random import shuffle
from timeit import default_timer as timer

DATA = {}

In [2]:
def kill(b):
    l = [
        [int(''.join(b[:9]), 2), int(b[27], 2)],
        [int(''.join(b[9:18]), 2), int(b[28], 2)],
        [int(''.join(b[18:27]), 2), int(b[29], 2)]
    ]
    for n in range(3):
        if l[n][1] == 1:
            l[n][0] = 0
    a = ''.join([str(i[1]) for i in l]) + b[30]
    return list('{0:09b}'.format(l[0][0]) +
                '{0:09b}'.format(l[1][0]) + 
                '{0:09b}'.format(l[2][0]) + 
                '{0:04b}'.format(int(a, 2)))

def move(b, m):
    n, x, y = m
    s = list(reversed('{0:031b}'.format(b | (2 ** ((n * 3 * 3) + (y * 3) + x)))))
    if(s[30] == '0'):
        s[30] = '1'
    else:
        s[30] = '0'
    for n in [i - 27 for i in [27, 28, 29] if s[i] == '0']:
        l, r = True, True
        for y_x in range(3): 
            x, y = True, True
            for x_y in range(3):
                if(s[(n * 3 * 3) + (y_x * 3) + x_y] == '0'): 
                    x = False
                if(s[(n * 3 * 3) + (x_y * 3) + y_x] == '0'): 
                    y = False

            if(y or x):
                s[n + 27] = '1'
                s = kill(s)
            if(s[(n * 3 * 3) + (y_x * 3) + y_x] == '0'): 
                l = False
            if(s[(n * 3 * 3) + (y_x * 3) + 2 - y_x] == '0'): 
                r = False
        if(l or r):
            s[n + 27] = '1'
            s = kill(s)


    return int(''.join(reversed(s)), 2)


def display(b):
    s = list(reversed('{0:031b}'.format(b)))
    print('Turn: {}'.format(s[30]))
    for n in range(3):
        print('Board {}:'.format(n + 1))
        for y in range(3):
            output = ''
            for x in range(3):
                output += s[(n * 3 * 3) + (y * 3) + x] + ' '
            print(output + '\n')
    print('Dead: {} {} {}'.format(s[27], s[28], s[29]))


def possible(b):
    s = list(reversed('{0:031b}'.format(b)))
    p = []
    for n in [i - 27 for i in [27, 28, 29] if s[i] == '0']:
        for y in range(3):
            for x in range(3):
                if(s[(n * 3 * 3) + (y * 3) + x] == '0'): 
                    p.append([n, x, y]) 
    return p

In [3]:
def value(node, alpha, beta, player):
        if len(node.children) == 0:
            return node.value()
        if node.saved_value:
            return node.saved_value
        if player:
            v = -1000;
            for n in node.children:
                v = max(v, value(DATA[n], alpha, beta, False))
                if v > alpha:
                    alpha = v
                if beta <= alpha:
                    break
                node.saved_value = alpha
            return alpha;
        else:
            v = 1000;
            for n in node.children:
                v = min(v, value(DATA[n], alpha, beta, True))
                if(v < beta):
                    beta = v
                if (beta <= alpha):
                    break
                node.saved_value = beta
            return beta;

In [4]:
def translate(b):
    # Rotate boards
    c = [[0, 1, 3], [1, 2, 5], [5, 7, 8], [3, 6, 7]]
    r = [2, 5, 8, 1, 4, 7, 0, 3, 6]

    for n in range(3):
        if int(''.join(b[n * 9 : 9 + 9 * n]), 2) == 0:
            continue
        v = []
        for N, u in enumerate(c):
            v.append([0, 0, 0, 0])
            for i, corner in enumerate(u):
                v[N][i] += 1 if b[n * 9 + corner] == '1' else 0
        v = [sum(i) for i in v]
        to = v.index(max(v))
        for _ in range(to):
            old = list(b[(n * 9):(9 + n * 9)])
            for element in range(9):
                b[element + n * 9] = old[r[element]]

    l = [
        [int(''.join(b[:9]), 2), int(b[27], 2)],
        [int(''.join(b[9:18]), 2), int(b[28], 2)],
        [int(''.join(b[18:27]), 2), int(b[29], 2)]
    ]
    # Sort boards
    for n in range(2):
        if l[n][1] == 1:
            continue
        for k in range(n + 1, 3):
            if l[k][1] == 1:
                continue
            if l[n] < l[k]:
                temp = list(l[n])
                l[n] = list(l[k])
                l[k] = temp

    a = ''.join([str(i[1]) for i in l]) + b[30]
    d = ''.join(list(reversed('{0:09b}'.format(l[0][0]) +
                              '{0:09b}'.format(l[1][0]) +
                              '{0:09b}'.format(l[2][0]) +
                              '{0:04b}'.format(int(a, 2)))))
    return int(d, 2)

In [7]:
class Node:
    def __init__(self, board):
        self.board = board
        self.children = []
        self.saved_value = None

    def expand(self):
        s = '{0:031b}'.format(self.board)
        if len([i for i in [1, 2, 3] if s[i] == '0']) > 0:
            for m in possible(self.board):
                b = list(reversed('{0:031b}'.format(move(self.board, m))))
                #b = translate(b)
                b = int(''.join(list(reversed(b))), 2)
                if not b in DATA:
                    n = Node(b)
                    DATA[n.board] = n
                    queue.put(n)
                self.children.append(b)

    def value(self):
        if '{0:031b}'.format(self.board)[0] == '0':
            return 1
        else:
            return -1

In [8]:
DATA = {}
BASE = int('0110000000000000000000000000000', 2)
root = Node(BASE)
queue = Queue()
queue.put(root)

n, p = 1, 1

start = timer()

while not queue.empty():
    queue.get().expand()
    if n % 10000 == 0:
        print('{}\tof {}\tIn queue: {}\t\tAverage BF: {}\tTime: {}'.format(
            n, 
            len(DATA), queue.qsize(), 
            '{0:.2f}'.format(1 +(queue.qsize() - p) / 10000), 
            '{0:.2f}'.format(timer() - start))
        )
        p = queue.qsize() 
    n += 1
    
print('{} of {} done after {}.'.format(n - 1, len(DATA), queue.qsize(), timer() - start))

232 of 231 done after 0.


In [9]:
RESULT = {}

for i, c in enumerate(root.children):
    if not c in RESULT:
        print('{} of {}'.format(i + 1, len(root.children)))
        RESULT[c] = value(DATA[c], -1000, 1000, True)
    else:
        print('{} of {} (Cached)'.format(i + 1, len(root.children)))
        
RESULT

1 of 9
2 of 9
3 of 9
4 of 9
5 of 9
6 of 9
7 of 9
8 of 9
9 of 9


{1879048193: 1,
 1879048194: -1,
 1879048196: -1,
 1879048200: 1,
 1879048208: 1,
 1879048224: 1,
 1879048256: 1,
 1879048320: 1,
 1879048448: 1}

In [10]:
 timer() - start

6.5974006410106085

In [11]:
for key in RESULT:
    display(key)
    print(RESULT[key])

Turn: 1
Board 1:
1 0 0 

0 0 0 

0 0 0 

Board 2:
0 0 0 

0 0 0 

0 0 0 

Board 3:
0 0 0 

0 0 0 

0 0 0 

Dead: 0 1 1
1
Turn: 1
Board 1:
0 1 0 

0 0 0 

0 0 0 

Board 2:
0 0 0 

0 0 0 

0 0 0 

Board 3:
0 0 0 

0 0 0 

0 0 0 

Dead: 0 1 1
-1
Turn: 1
Board 1:
0 0 1 

0 0 0 

0 0 0 

Board 2:
0 0 0 

0 0 0 

0 0 0 

Board 3:
0 0 0 

0 0 0 

0 0 0 

Dead: 0 1 1
-1
Turn: 1
Board 1:
0 0 0 

1 0 0 

0 0 0 

Board 2:
0 0 0 

0 0 0 

0 0 0 

Board 3:
0 0 0 

0 0 0 

0 0 0 

Dead: 0 1 1
1
Turn: 1
Board 1:
0 0 0 

0 1 0 

0 0 0 

Board 2:
0 0 0 

0 0 0 

0 0 0 

Board 3:
0 0 0 

0 0 0 

0 0 0 

Dead: 0 1 1
1
Turn: 1
Board 1:
0 0 0 

0 0 1 

0 0 0 

Board 2:
0 0 0 

0 0 0 

0 0 0 

Board 3:
0 0 0 

0 0 0 

0 0 0 

Dead: 0 1 1
1
Turn: 1
Board 1:
0 0 0 

0 0 0 

1 0 0 

Board 2:
0 0 0 

0 0 0 

0 0 0 

Board 3:
0 0 0 

0 0 0 

0 0 0 

Dead: 0 1 1
1
Turn: 1
Board 1:
0 0 0 

0 0 0 

0 1 0 

Board 2:
0 0 0 

0 0 0 

0 0 0 

Board 3:
0 0 0 

0 0 0 

0 0 0 

Dead: 0 1 1
1
Turn: 1
Board 1:
0 0 0 

0 0 0

In [ ]:
current = root
player = True
while len(current.children) > 0:
    display(current.board)
    values = [value(DATA[o], -1000, 1000, player) for o in current.children]
    neste = DATA[current.children[values.index(max(values) if player else min(values))]]
    print(max(values) if player else min(values))
    player = not player
    current = neste